# Семинар 9: Character-Level LSTM

## Вступление
На прошлом занятии мы познакомились с тем, как можно векторизовать текстовые данные для решения задач обработки текстов. Сегодня мы продолжим заниматься текстами и посмотрим на простейший пример автоматической генерации текстов при помощи Recurrent Neural Network (RNN).

Полезные материалы по RNN можно почитать [здесь]((http://karpathy.github.io/2015/05/21/rnn-effectiveness/)), а реализацию на PyTorch — [здесь](https://github.com/karpathy/char-rnn).

### План семинара
1. Подготовка данных
2. Имплементация модели
3. Обучение модели
4. Применение модели

In [1]:
from typing import Iterable, Tuple

import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

## 1. Подготовка данных

### Загрузим текст "Анны Карениной"

In [2]:
with open("anna.txt", "r") as f:
    text = f.read()

text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Токенизируем текст

Аналогично предыдущему семинару, в ячейках ниже создадим два **словаря** для преобразования символов в целые числа и обратно.

In [3]:
unique_chars = tuple(set(text))
int2char = dict(enumerate(unique_chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])
encoded[:100]

array([15, 76, 14, 56, 68, 46, 71, 80, 70, 72, 72, 72,  3, 14, 56, 56, 67,
       80, 55, 14, 69,  1, 45,  1, 46, 66, 80, 14, 71, 46, 80, 14, 45, 45,
       80, 14, 45,  1, 74, 46, 52, 80, 46, 23, 46, 71, 67, 80, 59, 44, 76,
       14, 56, 56, 67, 80, 55, 14, 69,  1, 45, 67, 80,  1, 66, 80, 59, 44,
       76, 14, 56, 56, 67, 80,  1, 44, 80,  1, 68, 66, 80, 28,  7, 44, 72,
        7, 14, 67, 48, 72, 72,  2, 23, 46, 71, 67, 68, 76,  1, 44])

Посмотрим на схему char-RNN:
<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charseq.jpeg?raw=1" width="30%">

Сеть ожидает **one-hot encoded** входа, что означает, что каждый символ преобразуется в целое число (через созданный маппинг), а затем преобразуется в вектор-столбец, где только соответствующий ему целочисленный индекс будет иметь значение 1, а остальная часть вектора будет заполнена нулями. Давайте напишем функцию для этого преобразования.

#### Задание: допишите функцию one-hot кодирования последовательности

In [4]:
def one_hot_encode(int_words: np.array, n_labels: int) -> np.array:
    """
    Creates one-hot representation matrix for a given batch of integer sequences
    :param int_words: array of ints, which represents current sequence; shape: [batch_size, seq_len]
    :param n_labels: vocabulary size (number of unique tokens in data)
    :return: one-hot representation of the input array; shape: [batch_size, seq_len, n_labels]
    """
    words_one_hot = np.zeros((int_words.size, n_labels), dtype=np.float32)
    words_one_hot[np.arange(words_one_hot.shape[0]), int_words.flatten()] = 1.
    words_one_hot = words_one_hot.reshape((*int_words.shape, n_labels))

    return words_one_hot

In [5]:
# testing the function
test_seq = np.array([[3, 5, 1], [0, 2, 4]])
test_one_hot = one_hot_encode(test_seq, 8)

print(test_one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0.]]]


### Сформируем батчи
На простом примере батчи будут выглядеть так: мы возьмем закодированные символы и разделим их на несколько последовательностей, заданных параметром `batch_size`. Каждая из наших последовательностей будет иметь длину `seq_length`.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>

**1. Отбросим часть текста, чтобы у нас были только полные батчи**

Каждый батч содержит $N \times M$ символов, где $N$ — это количество последовательностей в батче (`batch_size`), а $M$ — длина каждой последовательности (`seq_length`). Затем, чтобы получить общее количество батчей $K$, которое мы можем сделать из последовательности, нужно разделить длину последовательности на количество символов в батче. Когда мы узнаем количество батчей, можно получить общее количество символов, которые нужно сохранить, из последовательности: $N \times M \times K$.

**2. Разделим текст на $N$ частей**

Этот шаг нужен, чтобы мы могли проходить по тексту окном размера `[batch_size, seq_len]`. Его можно реализовать при помощи простого `reshape`.

**3. Теперь, когда у нас готова матрица текста, мы можем двигаться по ней окном его, чтобы получить батчи**

Из каждой позиции окна сформируем обучающие пары `(x, y)` следующим образом: $x$ — это все элементы окна кроме последнего столбца, а $y$ — это все элементы окна кроме первого столбца. Тем самым для каждого токена исходного текста мы будем предсказывать следующий за ним токен.

#### Задание: допишите функцию генерации батчей

In [18]:
def get_batches(int_words: np.array, batch_size: int, seq_length: int) -> Iterable[np.array]:
    """
    Generates batches from encoded sequence.
    :param int_words: array of ints, which represents the text; shape: [batch_size, -1]
    :param batch_size: number of sequences per batch
    :param seq_length: number of encoded chars in a sequence
    :return: generator of pairs (x, y); x_shape, y_shape: [batch_size, seq_length - 1]
    """
    # 1. Truncate text, so there are only full batches
    window_size = seq_length + 1
    batch_size_total = batch_size * window_size
    n_batches = len(int_words) // batch_size_total
    int_words = int_words[:n_batches * batch_size_total]

    # 2. Reshape into batch_size rows
    int_words = int_words.reshape((batch_size, -1))
    
    # 3. Iterate through the text matrix
    for position in range(0, int_words.shape[1], window_size):
        x = int_words[:, position:position + window_size - 1]
        y = int_words[:, position + 1:position + window_size]
        # try:
        #     y[:, :-1], y[:, -1] = x[:, 1:], int_words[:, position + seq_length]
        # except IndexError:
        #     y[:, :-1], y[:, -1] = x[:, 1:], int_words[:, 0]
        yield x, y

In [19]:
# testing the function
test_batches = get_batches(encoded, 8, 50)
test_x, test_y = next(test_batches)
assert test_x.shape == test_y.shape
print(f"x:\n{test_x[:10, :10]}\n")
print(f"y:\n{test_y[:10, :10]}")

x:
[[15 76 14 56 68 46 71 80 70 72]
 [56 56 46 58 80 14 44 58 80 66]
 [68 76 14 68 80  1 44 66 28 45]
 [46  7 80 68 76 28 59 39 76 68]
 [46 71 63 80 76 46 80 76 14 58]
 [71 63 80  7 76 28 80  7 14 66]
 [66 68 80 13 46 80 34 28 23 46]
 [63 80 25 45 46 77 46 67 80 25]]

y:
[[76 14 56 68 46 71 80 70 72 72]
 [56 46 58 80 14 44 58 80 66 76]
 [76 14 68 80  1 44 66 28 45 59]
 [ 7 80 68 76 28 59 39 76 68 55]
 [71 63 80 76 46 80 76 14 58 80]
 [63 80  7 76 28 80  7 14 66 80]
 [68 80 13 46 80 34 28 23 46 71]
 [80 25 45 46 77 46 67 80 25 45]]


## 2. Имплементация модели

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charRNN.png?raw=1" width=50%>

### Структура модели

* Создаём и храним необходимые словари.
* Определяем слой [LSTM]((https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM)) с помощью инстанса класса `torch.nn.LSTM`, который принимает набор параметров: `input_size` — длина последовательности в батче; `n_hidden` — размер скрытых слоёв; `n_layers` — количество слоёв; `drop_prob` — вероятность дропаута; и `batch_first` — флаг, указывающий на то, что у входных последовательностей размерность батча идёт вдоль нулевой оси.
* Определяем слой Dropout с таким же значением `drop_prob`.
* Определяем полносвязный слой с набором параметров: размерность ввода — `n_hidden`; размерность выхода — размер словаря.
* Наконец, инициализируем веса и начальное скрытое состояние (`self.init_hidden()`).

In [20]:
class CharRNN(nn.Module):
    def __init__(
        self,
        unique_tokens: Tuple[str],
        n_hidden: int = 256,
        n_layers: int = 2,
        drop_prob: float = 0.5,
        lr: float = 0.001
    ) -> None:
        super().__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.drop_prob = drop_prob
        self.lr = lr
        
        # creating mappings
        self.unique_tokens = unique_tokens
        self.int2char = dict(enumerate(self.unique_tokens))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM, dropout and fully connected layers
        self.lstm = nn.LSTM(len(self.unique_tokens), n_hidden, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, len(self.unique_tokens))

    def forward(self, x: torch.Tensor, hidden: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        # Stack up LSTM outputs using view. You may need to use contiguous to reshape the output.
        out = out.contiguous().view(-1, self.n_hidden)
        ## Get the output for classification.
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size: int, weight_device: torch.device) -> Tuple[torch.Tensor]:
        """
        Creates two new zero tensors for hidden state and cell state of LSTM
        :param batch_size: number of sequences per batch
        :param weight_device: torch.device("cuda") for GPU init or torch.device("cpu") for CPU init
        :return: tuple of two tensors of shape [n_layers x batch_size x n_hidden]
        """
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(weight_device),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(weight_device))
        return hidden

## 3. Обучение модели

По классике, используем оптимизатор Adam и кросс-энтропию. Но без пары особенностей не обойтись:
* Во время цикла будем отделять скрытое состояние от его истории, потому что скрытое состояние LSTM, является кортежем скрытых состояний.
* Будем использовать [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html), чтобы избавиться от взрывающихся градиентов.

In [21]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [28]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    net.train()
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    net.to(device)
    
    counter = 0
    n_chars = len(net.unique_tokens)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size, device)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x).to(device), torch.from_numpy(y).to(device)

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.reshape(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size, device)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    inputs, targets = torch.from_numpy(x).to(device), torch.from_numpy(y).to(device)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.reshape(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train()
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [29]:
net = CharRNN(unique_chars, n_hidden=32, n_layers=2)
train(net, encoded, epochs=20, batch_size=128, seq_length=100, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 4.3006... Val Loss: 4.2894
Epoch: 1/20... Step: 20... Loss: 4.1379... Val Loss: 4.1019
Epoch: 1/20... Step: 30... Loss: 3.7318... Val Loss: 3.6390
Epoch: 1/20... Step: 40... Loss: 3.4612... Val Loss: 3.3511
Epoch: 1/20... Step: 50... Loss: 3.3512... Val Loss: 3.2270
Epoch: 1/20... Step: 60... Loss: 3.2861... Val Loss: 3.1733
Epoch: 1/20... Step: 70... Loss: 3.2779... Val Loss: 3.1501
Epoch: 1/20... Step: 80... Loss: 3.2537... Val Loss: 3.1387
Epoch: 1/20... Step: 90... Loss: 3.2321... Val Loss: 3.1329
Epoch: 1/20... Step: 100... Loss: 3.1972... Val Loss: 3.1301
Epoch: 1/20... Step: 110... Loss: 3.2012... Val Loss: 3.1280
Epoch: 1/20... Step: 120... Loss: 3.2104... Val Loss: 3.1264
Epoch: 1/20... Step: 130... Loss: 3.1765... Val Loss: 3.1251
Epoch: 2/20... Step: 140... Loss: 3.1581... Val Loss: 3.1241
Epoch: 2/20... Step: 150... Loss: 3.1716... Val Loss: 3.1237
Epoch: 2/20... Step: 160... Loss: 3.1630... Val Loss: 3.1232
Epoch: 2/20... Step: 170... Loss:

## 4. Применение модели

Сперва сохраним обученную модель, чтобы можно было загрузить её позже. В следующей ячейке сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметры скрытого слоя и токены.

In [30]:
checkpoint = {"n_hidden": net.n_hidden,
              "n_layers": net.n_layers,
              "state_dict": net.state_dict(),
              "tokens": net.unique_tokens}

with open("rnn_x_epoch.net", "wb") as f:
    torch.save(checkpoint, f)

### Делаем предсказания

Сгенерируем текст! Для предсказания продолжения текста мы передаём в сеть последний символ, она предсказывает следующий символ, который мы снова передаем на вход, получаем ещё один предсказанный символ и так далее. Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным символам. Мы можем ограничить число символов на каждом шаге генерации, чтобы сделать получаемый предсказанный текст более разумным, рассматривая только некоторые, наиболее вероятные символы. С одной стороны, такой подход позволит нам рассматривать не только самую вероятную последовательность с точки зрения прогноза модели. С другой стороны, мы будем работать с ограниченным набором сгенерированных вариантов, поэтому избавимся от совсем уж шумовых прогнозов. Узнать больше можно [здесь](https://pytorch.org/docs/stable/generated/torch.topk.html#torch.topk).

In [38]:
def predict_next_char(net: torch.nn.Module, char: str, h: torch.Tensor = None, top_k: int = None) -> Tuple[str, torch.Tensor]:
        """
        Given a character and a model, predicts next character in the sequence
        :param net:
        :param char:
        :param h:
        :param top_k:
        :return:
        """
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.unique_tokens))
        inputs = torch.from_numpy(x).to(device)

        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data.cpu()

        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.unique_tokens))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming и генерирование текста

Нужно задать скрытое состояние, чтобы сеть не генерировала произвольные символы. 

In [39]:
def sample(net, size, prime="The", top_k=None):
    net.to(device)
    net.eval()
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1, device)
    for ch in prime:
        char, h = predict_next_char(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict_next_char(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return "".join(chars)

### Загрузка чекпоинта и генерация

In [40]:
with open("rnn_x_epoch.net", "rb") as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint["tokens"], n_hidden=checkpoint["n_hidden"], n_layers=checkpoint["n_layers"])
loaded.load_state_dict(checkpoint["state_dict"])

# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said hes had tardes he adder asten ang ther tan has that sinter sare the to seed tantare to the ther tould she wons,
titite thad ans,
an to to than whout he than to soot
had shas her. In sher ting
he adt the att hares
and woul hit ase,
thit ottel houd, to, his tisilind the arset the, house hes, har ad ard thot at adsisthint hor thound, hir salt,," shid,, wesins he wos ot
hore at to woust the sale the hesens othes wend whas wees the
hertes to she
she her seen of,, wout thy at ho the sat the hose wont we hor ant to a he thoet the wist. Is the, tho was wat wile teer heed the sertas sore so as tangad the telt, white an shing sore ther horsad at sorend hins one hes whonilt ad ass hir thint, wan so the tored
was, she anget he sisten shit to
an of so the wat sat adsind so her tho he
hand her hat anse so soos thit,
her and, hot had tand,
the, was, an ate sast tart she to tas shad, sit the adlint ang at, he ant at
hat tas tine shose hanse he wese hon as sore to so as this sittitt so a